In [ ]:
import pandas as pd
import requests

!pip install sqlalchemy
!pip install pymysql

In [ ]:
schema = "EuroEnergy"
host = "127.0.0.1"
user = "root"
password = "YOUR PASSWORD"
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [ ]:
 weather_data = pd.read_csv('all_cities_df.csv')
 weather_data.head()

city_coordinates = pd.read_csv('city_coordinates.csv')
city_coordinates = city_coordinates.drop(columns = 'Unnamed: 0').copy()

weather_data.info()
city_coordinates

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82185 entries, 0 to 82184
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  82185 non-null  int64  
 1   datetime    82185 non-null  object 
 2   tempmax     82185 non-null  float64
 3   tempmin     82185 non-null  float64
 4   temp        82185 non-null  float64
 5   humidity    82185 non-null  float64
 6   precip      82185 non-null  float64
 7   windspeed   82185 non-null  float64
 8   cloudcover  82185 non-null  float64
 9   conditions  82185 non-null  object 
 10  city        82185 non-null  object 
 11  lat         82185 non-null  float64
 12  lon         82185 non-null  float64
dtypes: float64(9), int64(1), object(3)
memory usage: 8.2+ MB


,city,lat,lon,country_code
0,Berlin,52.517037,13.388860,DE
1,Dortmund,51.514227,7.465279,DE
2,Kassel,51.315455,9.492410,DE
3,Munich,48.137108,11.575382,DE
4,Madrid,40.416705,-3.703582,ES
5,Bilbao,43.263002,-2.935004,ES
6,Seville,37.388630,-5.995340,ES
7,Valencia,39.469707,-0.376335,ES
8,Oslo,59.913330,10.738970,NO
9,Trondheim,63.430566,10.395193,NO


In [ ]:
country_info = pd.read_sql('countries', con=connection_string)

In [ ]:
country_info


,country_id,country_name
0,1,Germany
1,2,Spain
2,3,Norway
3,4,Slovak Republic


In [ ]:

#country_map = {'DE':'Germany', 'ES': 'Spain', 'NO':'Norway', 'SK': 'Slovak Republic'
city_coordinates['country'] = ''

city_coordinates['country'] = ''  # Initialize empty string column

for idx, row in city_coordinates.iterrows():
    code = row['country_code']
    if code == 'DE':
        city_coordinates.at[idx, 'country'] = 'Germany'
    elif code == 'ES':
        city_coordinates.at[idx, 'country'] = 'Spain'
    elif code == 'NO':
        city_coordinates.at[idx, 'country'] = 'Norway'
    elif code == 'SK':
        city_coordinates.at[idx, 'country'] = 'Slovak Republic'

In [ ]:
country_cities = city_coordinates.merge(country_info, left_on= 'country',
                                        right_on='country_name',
                                        how = 'left').drop(axis = 0, columns ='country')
country_cities

,city,lat,lon,country_code,country_id,country_name
0,Berlin,52.517037,13.388860,DE,1,Germany
1,Dortmund,51.514227,7.465279,DE,1,Germany
2,Kassel,51.315455,9.492410,DE,1,Germany
3,Munich,48.137108,11.575382,DE,1,Germany
4,Madrid,40.416705,-3.703582,ES,2,Spain
5,Bilbao,43.263002,-2.935004,ES,2,Spain
6,Seville,37.388630,-5.995340,ES,2,Spain
7,Valencia,39.469707,-0.376335,ES,2,Spain
8,Oslo,59.913330,10.738970,NO,3,Norway
9,Trondheim,63.430566,10.395193,NO,3,Norway


# MySQL code

USE EuroEnergy;

CREATE TABLE country_cities (
	city_id INT AUTO_INCREMENT,
    city VARCHAR (250) NOT NULL,
    lat FLOAT NOT NULL,
    lon FLOAT NOT NULL,
    country_code VARCHAR(3)NOT NULL,
    country_id INT NOT NULL,
    country_name VARCHAR(250) NOT NULL,
    PRIMARY KEY (city_id),
    FOREIGN KEY (country_id) References countries(country_id)
    );
    

In [ ]:
country_cities.to_sql('country_cities',
                  if_exists='append',
                  con=connection_string,
                  index=False)

15

In [ ]:
country_cities = pd.read_sql('country_cities', con = connection_string)

country_cities

,city_id,city,lat,lon,country_code,country_id,country_name
0,1,Berlin,52.5170,13.388900,DE,1,Germany
1,2,Dortmund,51.5142,7.465280,DE,1,Germany
2,3,Kassel,51.3155,9.492410,DE,1,Germany
3,4,Munich,48.1371,11.575400,DE,1,Germany
4,5,Madrid,40.4167,-3.703580,ES,2,Spain
5,6,Bilbao,43.2630,-2.935000,ES,2,Spain
6,7,Seville,37.3886,-5.995340,ES,2,Spain
7,8,Valencia,39.4697,-0.376335,ES,2,Spain
8,9,Oslo,59.9133,10.739000,NO,3,Norway
9,10,Trondheim,63.4306,10.395200,NO,3,Norway


In [ ]:

 country_weather_data = weather_data.merge(country_cities, left_on = ['lat','lon'],
                    right_on=['lat','lon'],
                    how = 'left').drop(axis = 0, columns = 'country_code')
 country_weather_data = country_weather_data.drop( axis= 0, columns = ['city_x']).copy()
 country_weather_data = country_weather_data.rename( columns = {'datetime':'datetime',
                'tempmax':'tempmax','tempmin':'tempmin',
                'temp':'temp','humidity':'humidity', 'precip':'precip', 'windspeed':'windspeed',
            'cloudcover':'cloudcover', 'conditions':'conditions', 'lat':'lat',
        'lon':'lon', 'city_y':'city', 'country_id':'country_id',
        'country_name':'country_name'})

country_weather_data

,datetime,tempmax,tempmin,temp,humidity,precip,windspeed,cloudcover,conditions,lat,lon,city,country_id,country_name
0,2010-01-01,-1.0,-3.0,-1.8,86.7,3.2,23.2,92.5,"Snow, Rain, Overcast",52.517037,13.388860,Berlin,1,Germany
1,2010-01-02,-1.0,-4.0,-2.0,89.3,1.8,19.5,92.4,"Snow, Rain, Overcast",52.517037,13.388860,Berlin,1,Germany
2,2010-01-03,-4.0,-7.0,-4.6,93.1,0.2,12.1,92.9,"Snow, Overcast",52.517037,13.388860,Berlin,1,Germany
3,2010-01-04,-5.0,-8.0,-6.5,93.8,0.0,19.5,93.6,Overcast,52.517037,13.388860,Berlin,1,Germany
4,2010-01-05,-5.0,-11.0,-6.7,86.8,0.0,15.7,79.2,Partially cloudy,52.517037,13.388860,Berlin,1,Germany
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82180,2024-12-27,5.1,-4.0,-0.7,72.7,0.0,15.8,17.5,Clear,49.223467,18.739314,Žilina,4,Slovak Republic
82181,2024-12-28,7.3,-2.0,0.9,80.4,0.0,14.7,40.1,Partially cloudy,49.223467,18.739314,Žilina,4,Slovak Republic
82182,2024-12-29,5.2,-5.0,-1.6,90.1,0.0,4.1,27.4,Partially cloudy,49.223467,18.739314,Žilina,4,Slovak Republic
82183,2024-12-30,-1.9,-4.0,-2.8,95.9,0.0,4.4,95.8,Overcast,49.223467,18.739314,Žilina,4,Slovak Republic


In [ ]:
#country_weather_data.to_csv('daily_weather_data.csv')
country_weather_data = pd.read_csv('daily_weather_data.csv')

In [ ]:
weather_data['lat'] = weather_data['lat'].round(2)
weather_data['lon'] = weather_data['lon'].round(2)

country_cities['lat'] = country_cities['lat'].round(2)
country_cities['lon'] = country_cities['lon'].round(2)

In [ ]:
country_cities

,city,lat,lon,country_code,country_id,country_name
0,Berlin,52.52,13.39,DE,1,Germany
1,Dortmund,51.51,7.47,DE,1,Germany
2,Kassel,51.32,9.49,DE,1,Germany
3,Munich,48.14,11.58,DE,1,Germany
4,Madrid,40.42,-3.70,ES,2,Spain
5,Bilbao,43.26,-2.94,ES,2,Spain
6,Seville,37.39,-6.00,ES,2,Spain
7,Valencia,39.47,-0.38,ES,2,Spain
8,Oslo,59.91,10.74,NO,3,Norway
9,Trondheim,63.43,10.40,NO,3,Norway


In [ ]:
 country_weather_data.isna().sum()


Unnamed: 0      0
datetime        0
tempmax         0
tempmin         0
temp            0
humidity        0
precip          0
windspeed       0
cloudcover      0
conditions      0
lat             0
lon             0
city            0
country_id      0
country_name    0
dtype: int64

In [ ]:
#country_weather_data['city'].unique()

country_weather_data['datetime'] = pd.to_datetime(country_weather_data['datetime'])
country_weather_data['year_month'] = country_weather_data['datetime'].dt.to_period('M')

country_weather_data['year_month'].unique()


In [ ]:
country_weather_data['year_month'] = country_weather_data['datetime'].dt.to_period('M')

country_weather_data['year_month'].unique()

<PeriodArray>
['2010-01', '2010-02', '2010-03', '2010-04', '2010-05', '2010-06', '2010-07',
 '2010-08', '2010-09', '2010-10',
 ...
 '2024-03', '2024-04', '2024-05', '2024-06', '2024-07', '2024-08', '2024-09',
 '2024-10', '2024-11', '2024-12']
Length: 180, dtype: period[M]

In [ ]:
#monthly weather aggergates
monthly_stats = (
    country_weather_data.groupby(['year_month', 'city','country_name'])
      .agg(
          avg_temp=('temp', 'mean'),
          # most_temp = ('temp', 'mode'),
          min_temp=('tempmin', 'min'),
                  max_temp=('tempmax', 'max'),
                  max_windspeed = ('windspeed', 'max'),
                  avg_windspeed = ('windspeed', 'mean'),
                  avg_cloudcover = ('cloudcover', 'mean'),

                  total_precip=('precip', 'sum'),
                  #dominant_temp_range=('temp_range', lambda x: x.mode()[0])
      )
      .reset_index()
)


In [ ]:
Country_monthly_weather.to_csv('monthly_weather_stats.csv')

In [ ]:
#Country_monthly_weather = pd.read_csv('monthly_weather_stats.csv')
Country_monthly_weather = Country_monthly_weather.drop(axis = 0, columns = 'Unnamed: 0').copy()
# monthly_stats
# country_cities
Country_monthly_weather

,year_month,city,country_name,avg_temp,min_temp,max_temp,max_windspeed,avg_windspeed,avg_cloudcover,total_precip,city_id,country_code,country_id
0,2010-01,Alta,Norway,-8.290323,-23.9,6.7,58.4,25.670968,63.087097,1.787,12,NO,3
1,2010-01,Berlin,Germany,-5.025806,-19.0,2.0,50.0,23.274194,79.251613,48.300,1,DE,1
2,2010-01,Bilbao,Spain,6.819355,-12.0,19.0,60.9,23.400000,73.903226,198.184,6,ES,2
3,2010-01,Bratislava,Slovak Republic,-2.516129,-14.8,8.8,38.7,20.064516,78.112903,54.255,13,SK,4
4,2010-01,Dortmund,Germany,-1.606452,-12.7,5.5,39.6,19.038710,79.770968,53.400,2,DE,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2695,2024-12,Oslo,Norway,0.458065,-8.0,9.0,38.4,21.438710,72.074194,47.022,9,NO,3
2696,2024-12,Seville,Spain,11.070968,2.1,21.7,32.5,17.593548,27.835484,4.367,7,ES,2
2697,2024-12,Trondheim,Norway,1.364516,-8.0,12.0,66.0,26.116129,82.574194,130.966,10,NO,3
2698,2024-12,Valencia,Spain,11.538710,2.0,24.9,45.4,17.070968,36.267742,6.031,8,ES,2


In [ ]:
Country_monthly_weather = monthly_stats.merge(country_cities, on = ['city','country_name'],
                    how = 'left').drop(axis = 0, columns = ['lat','lon']).copy()

Next step, merge with country_table info to get country_id,
Send to MySQL
Set-energy data as period
Merge with Energy data using period, and country


In [ ]:
Energy_data = pd.read_csv('energy_population_data.csv')
Energy_data=Energy_data.drop(axis = 0, columns = 'Unnamed: 0').copy()
Energy_data

,country,date_of_energy_records,balance,product,value(GWh),year,population_id,population,Pecent_population_urban,population_recorded_in
0,Spain,2010-01-01,Net Electricity Production,"Total Renewables (Hydro, Geo, Solar, Wind, Other)",10143.8600,2010,23,46840500.0,78.4,2010
1,Spain,2010-01-01,Net Electricity Production,Not Specified,11.9170,2010,23,46840500.0,78.4,2010
2,Spain,2010-01-01,Net Electricity Production,Solar,273.6020,2010,23,46840500.0,78.4,2010
3,Spain,2010-01-01,Net Electricity Production,Other Combustible Non-Renewables,45.8970,2010,23,46840500.0,78.4,2010
4,Spain,2010-01-01,Net Electricity Production,Combustible Renewables,279.6320,2010,23,46840500.0,78.4,2010
...,...,...,...,...,...,...,...,...,...,...
13247,Norway,2024-12-01,Net Electricity Production,Electricity,15846.6010,2024,35,5576660.0,85.5,2024
13248,Germany,2024-12-01,Net Electricity Production,Hydro,2241.2938,2024,1,84552200.0,76.0,2024
13249,Germany,2024-12-01,Net Electricity Production,Electricity,42748.3323,2024,1,84552200.0,76.0,2024
13250,Germany,2024-12-01,Net Electricity Production,Wind,15408.7563,2024,1,84552200.0,76.0,2024


In [ ]:
Energy_data['date_of_energy_records'] = pd.to_datetime(Energy_data['date_of_energy_records'])
Energy_data['year_month'] = Energy_data['date_of_energy_records'].dt.to_period('M')

Energy_data['year_month'].unique()
# Country_monthly_weather['year_month']pd.to

<PeriodArray>
['2010-01', '2010-02', '2010-03', '2010-04', '2010-05', '2010-06', '2010-07',
 '2010-08', '2010-09', '2010-10',
 ...
 '2024-02', '2024-03', '2024-04', '2024-05', '2024-06', '2024-07', '2024-08',
 '2024-09', '2024-10', '2024-11']
Length: 180, dtype: period[M]

In [ ]:
Country_monthly_weather['year_month'] = (
    Country_monthly_weather['year_month'].astype('period[M]')
)

Energy_data['year_month'] = (
    Energy_data['year_month'].astype('period[M]')
)

In [ ]:
energy_weather_population = Country_monthly_weather.merge(Energy_data,
                              left_on = ['country_name', 'year_month'],
                              right_on = ['country', 'year_month'],
                              how = 'left').drop (axis = 0 ,
                                                  columns = ['country', 'year','Pecent_population_urban','country_code'])

In [ ]:
energy_weather_population.info()
energy_weather_population.to_csv('energy_weather_population.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49708 entries, 0 to 49707
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   year_month              49708 non-null  period[M]     
 1   city                    49708 non-null  object        
 2   country_name            49708 non-null  object        
 3   avg_temp                49708 non-null  float64       
 4   min_temp                49708 non-null  float64       
 5   max_temp                49708 non-null  float64       
 6   max_windspeed           49708 non-null  float64       
 7   avg_windspeed           49708 non-null  float64       
 8   avg_cloudcover          49708 non-null  float64       
 9   total_precip            49708 non-null  float64       
 10  city_id                 49708 non-null  int64         
 11  country_id              49708 non-null  int64         
 12  date_of_energy_records  49708 non-null  dateti